In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum

from main import Branch, Individual, Field


* Use SAR
  * Find relation between number of species and area
* Use EAR
  * Find relation between number of extinctions and decreasing area

In [ ]:
t = [-0.01, -0.03, -0.05, -0.10, -0.15, -0.20, -0.25, -0.30, -0.40, -0.50, -0.60, -0.65]
alpha = list(map(lambda o: 2**o, t))
species_alpha = [o for o in alpha for i in range(1)]

m = 14
l = 10
delta0 = 0.1
delta_diff = 8
d = 5

In [ ]:
grid = Field(
    species_alpha=species_alpha,
    m=m,
    l=l,
    delta0=delta0,
    delta_diff=delta_diff,
    d=d
)

In [ ]:
# species = grid.points
# for i in range(len(species)):
#     print(f"Species {i} with alpha={round(species_alpha[i], 3)} has {len(species[i])} individuals.")

In [ ]:
all_points = [p for species in grid.points for p in species]
x_coords = [p.x for p in all_points]
y_coords = [p.y for p in all_points]

x_min, x_max = min(x_coords), max(x_coords)
y_min, y_max = min(y_coords), max(y_coords)
origin = ((x_min + x_max)/2, (y_min + y_max)/2)

print(x_min, x_max, y_min, y_max)

In [ ]:
for processed_points in grid.points:
    x_scatter, y_scatter = zip(*map(lambda point: (point.x, point.y), processed_points))
    plt.scatter(x_scatter, y_scatter)
    plt.grid(True)
    plt.show()
# plt.savefig("./test.png")
# plt.clf()

In [ ]:
r_min = 0.5
r_max = 50.0
num_bins = 25
r_bins = np.logspace(np.log10(r_min), np.log10(r_max), num_bins + 1)
r_centers = 0.5 * (r_bins[:-1] + r_bins[1:])

In [ ]:
# gs = grid.all_pair_correlation(r_bins, l * l)
# plt.clf()
# plt.loglog(r_centers, gs, 'o-')
# plt.xlabel("r")
# plt.ylabel("g(r)")
# plt.grid(True)
# plt.show()
# # plt.savefig("rho.png")

In [ ]:
r = l / (2 ** (m/2))
r = 5.0


In [ ]:
# s_c = grid.s(r_max)

In [ ]:
# print(s_c)

In [ ]:
L_max = max(x_max - x_min, y_max - y_min)
L_list = np.logspace(np.log10(2*r), np.log10(L_max), 20)
A_list = [L**2 for L in L_list]

In [ ]:
# S_list = []

# for L in L_list:
#     S_L = grid.s(r, L, origin)
#     S_list.append(S_L)

In [ ]:
S_matrix = []
for _, species in enumerate(grid.points):
    S_list = []
    for L in L_list:
        origin = ((min(p.x for p in species) + max(p.x for p in species))/2,
                  (min(p.y for p in species) + max(p.y for p in species))/2)
        S_list.append(grid.s(r, L, origin))
    S_matrix.append(S_list)


In [ ]:
print(S_matrix)

In [ ]:
# plt.figure()
# plt.loglog(A_list, S_list, 'o-')
# plt.xlabel(r"log($A$)")
# plt.ylabel(r"log($S_C$)")
# plt.grid(True)
# plt.show()


In [ ]:
# # number of species within area A per alpha value
# for i, S_list in enumerate(S_matrix):
#     print(f"Alpha {i}:")
#     for S in range(len(A_list)):
#         print(f"  Area {round(A_list[S], 3)} : Species count {S_list[S]}")
        

In [ ]:
plt.figure(figsize=(6,5))

for i, S_list in enumerate(S_matrix):
    plt.plot(np.log10(A_list), np.log10(S_list), label=f'Alpha {i}')

plt.xlabel("log10(A)")
plt.ylabel("log10(S_c)")
plt.legend()
plt.grid(True)
plt.show()


Extinction-Area Relationship

In [ ]:
def determine_q(q_try, fractional_area, initial_species_count):
    lhs = fractional_area * initial_species_count
    lhs = np.full_like(q_try, lhs)
    rhs = (q_try / (1 - q_try)) - ((initial_species_count + 1) * q_try ** (initial_species_count + 1)) / (1 - q_try ** (initial_species_count + 1))
    root_find = lhs - rhs

    y_closest = np.min(np.abs(root_find))
    q_closest = q_try[np.argmin(np.abs(root_find))]
    print(f"Closest value to zero is {y_closest} at q={q_closest}")

    return q_closest

    # plt.figure()
    # plt.plot(q_try, root_find)
    # plt.xlabel('q')
    # plt.ylabel('Value')
    # plt.grid(True)
    # plt.show()

In [ ]:
q_try = np.linspace(0.99, 1.01, 10000000)
area_original = (x_max - x_min) * (y_max - y_min)
area_shrunk = (L_list[0])**2
fractional_area = area_shrunk / area_original
# species_initial = 1000
species_initial = len(all_points)
print(area_shrunk, area_original)
print(fractional_area, species_initial)
q = determine_q(q_try, fractional_area, species_initial)
print(q)

In [ ]:
def extinction_probability(a, q, n_c, n_0):
    print(a, q, n_c, n_0)
    return (q ** (n_c + 1) - 1) / (q ** (n_0 + 1) - 1)

In [ ]:
critical_abundance = 1000

extinction_prob = extinction_probability(a = fractional_area,
                                         q = q,
                                         n_c = critical_abundance,
                                         n_0 = species_initial)

print(f"Extinction probability: {extinction_prob}")